In [1]:
from django.db import connection 
from django.contrib.auth.models import User
from myboard import models

In [2]:
data = User.objects.all()

In [3]:
models.Board??

In [7]:
connection.queries[-1]

{'sql': 'SELECT "auth_user"."id", "auth_user"."password", "auth_user"."last_login", "auth_user"."is_superuser", "auth_user"."username", "auth_user"."first_name", "auth_user"."last_name", "auth_user"."email", "auth_user"."is_staff", "auth_user"."is_active", "auth_user"."date_joined" FROM "auth_user" LIMIT 21',
 'time': '0.000'}

In [69]:
User.objects.raw('SELECT max(age ) FROM myapp_user')

<RawQuerySet: SELECT max(age ) FROM myapp_user>

In [8]:
cursor = connection.cursor()
cursor.execute('SELECT max(age) from myapp_user')
max_value = cursor.fetchone()[0]

print(max_value)

None


In [11]:
cursor = connection.cursor()
cursor.execute('SELECT id, title, cnt from myboard_board')
results = cursor.fetchall()

In [11]:
print(type(results))

<class 'list'>


In [12]:
results[0]

(1, '자동차 검출44343', 5)

In [13]:
for r in results :
    print(r)

(1, '자동차 검출44343', 'fweew\r\n\r\nfwef\r\newew\r\nfwwe\r\nefwe\r\nf\r\newf\r\newfewfwefew', datetime.datetime(2020, 2, 19, 5, 8, 56, 774872), 5, None, 'common', 1)
(2, '32321321', '312312312', datetime.datetime(2020, 2, 19, 7, 39, 27, 299571), 0, None, 'common', 1)
(3, '32312323123', '32132133', datetime.datetime(2020, 2, 19, 7, 39, 54, 356455), 0, None, 'common', 1)
(4, '32312323123', '32132133', datetime.datetime(2020, 2, 19, 7, 39, 57, 463012), 0, None, 'common', 1)
(5, '3231231321', '3213123131231', datetime.datetime(2020, 2, 19, 7, 40, 10, 419637), 0, None, 'common', 1)
(6, '2424423424', '4242424\r\n24\r\n2\r\n42\r\n42\r\n4\r\n2', datetime.datetime(2020, 2, 19, 7, 47, 27, 154877), 0, None, 'common', 1)
(7, 'nnnbnnnnn', 'nnnnnn', datetime.datetime(2020, 2, 19, 7, 47, 59, 290686), 0, None, 'common', 1)
(8, '000000000000', '000000000\r\n0\r\n0\r\n0\r\n\r\n007777', datetime.datetime(2020, 2, 19, 7, 49, 9, 50042), 2, None, 'data', 1)


In [13]:
cursor.execute("""
select b.id, title, cnt, username 
from myboard_board b , auth_user u
where b.author_id = u.id and username = 'home'
""")

In [14]:
r = cursor.fetchone()

In [15]:
print(r)

(1, '자동차 검출44343', 5, 'home')


In [16]:
r = cursor.fetchone()
print(r)

(2, '32321321', 0, 'home')


In [18]:
r = cursor.fetchall()
print(r)


[]


In [22]:
cursor.execute("""
select b.id, title, cnt, username 
from myboard_board b , auth_user u
where b.author_id = u.id and username = 'home'
""")
result = cursor.fetchall()

In [23]:
for r in result :
    print(r[1])

자동차 검출44343
32321321
32312323123
32312323123
3231231321
2424423424
nnnbnnnnn
000000000000


In [24]:
cursor.description

(('id', None, None, None, None, None, None),
 ('title', None, None, None, None, None, None),
 ('cnt', None, None, None, None, None, None),
 ('username', None, None, None, None, None, None))

In [25]:
d = dict(zip(['a', 'b'],  (1,2) ))
print(d)

{'a': 1, 'b': 2}


In [33]:
z = zip(['a', 'b'],  (1,2) )
for v in z : 
    print(v)

('a', 1)
('b', 2)


In [27]:
d = dict(zip(['id', 'title', 'cnt', 'userame'],  result[0]  ))
print(d)

{'id': 1, 'title': '자동차 검출44343', 'cnt': 5, 'userame': 'home'}


In [34]:
def dictfetchall(cursor):    
    desc = cursor.description
    return [
        dict(zip([col[0] for col in desc], row))
        for row in cursor.fetchall()
    ]

In [37]:
cursor = connection.cursor()
cursor.execute('SELECT * from myboard_board')
d = dictfetchall(cursor)
print(d[0]["title"])


자동차 검출44343


In [ ]:
from django.shortcuts import render
from django.db import connection 
from django.test import RequestFactory
rf =  RequestFactory()

In [46]:
request = rf.get('common/5')

In [68]:
username = 'home'
category = 'common'
page = 2
sql = f"""
    select b.id, title, cnt, username 
    from myboard_board b , auth_user u
    where b.author_id = u.id and username = '{username}' and category = '{category}'
    """
print(sql)
cursor = connection.cursor()
cursor.execute(sql)


    select b.id, title, cnt, username 
    from myboard_board b , auth_user u
    where b.author_id = u.id and username = 'home' and category = 'common'
    


In [ ]:
data = dictfetchall(cursor)
data = data[(page-1)*3:page*3]

In [69]:
context = {"data":data}

print(context)

{'data': [{'id': 4, 'title': '32312323123', 'cnt': 0, 'username': 'home'}, {'id': 5, 'title': '3231231321', 'cnt': 0, 'username': 'home'}, {'id': 6, 'title': '2424423424', 'cnt': 0, 'username': 'home'}]}


In [70]:
response = render(request,  "myboard/list3.html", context)
print(response.content.decode())



32312323123 조횟수 0    home</a> <br>

3231231321 조횟수 0    home</a> <br>

2424423424 조횟수 0    home</a> <br>





In [72]:
def listsql(request, category, page)  :
    sql = f"""
    select b.id, title, cnt, username 
    from myboard_board b , auth_user u
    where b.author_id = u.id and username = '{username}' and category = '{category}'
    """
    print(sql)
    cursor = connection.cursor()
    cursor.execute(sql)
    
    data = dictfetchall(cursor)
    data = data[(page-1)*3:page*3]
    
    context = {"data":data}    
    
    return render(request,  "myboard/list3.html", context)   


In [73]:
#from myboard import views
#response = views.listsql(request, 'common', 2)
response = listsql(request, 'common', 2)

print(response.content.decode())


    select b.id, title, cnt, username 
    from myboard_board b , auth_user u
    where b.author_id = u.id and username = 'home' and category = 'common'
    


32312323123 조횟수 0    home</a> <br>

3231231321 조횟수 0    home</a> <br>

2424423424 조횟수 0    home</a> <br>





In [30]:
from django.test import RequestFactory
rf =  RequestFactory()
url = '/detailpost?pk=14'
my_request = rf.get(url)

In [31]:
from myapp import views
response = views.index(my_request)

In [34]:
print(type(response))


<class 'django.http.response.HttpResponse'>


In [37]:
print(response.content.decode())

tempalte h1^^^

<form action=/myformviewg2 method=post >
  <input type='hidden' name='csrfmiddlewaretoken' value='BzH4lfVleGfHTJTWJ59dlfYXZrtviUNfENNwqbhcM7mI9XD1VhOpJgFB3DeaIvA4' />
  <table>
  <tr><th><label for="id_age">나이:</label></th><td><input type="text" name="age" maxlength="3" required id="id_age" /></td></tr>
<tr><th><label for="id_memo">메모:</label></th><td><textarea name="memo" cols="40" rows="10" required id="id_memo">
</textarea></td></tr>
<tr><th><label for="id_hobby">취미:</label></th><td><input type="text" name="hobby" maxlength="4" required id="id_hobby" /></td></tr>
  </table>
  <input type=submit name=id value="ok">
</form>



In [75]:
username = "home"

sql = f"select id from auth_user where username='{username}'"

cursor.execute(sql)

author_id = cursor.fetchone()[0]
print(author_id)

1


In [82]:
filename = 'face2.png'

sql = f"""
INSERT INTO myboard_image
("author_id", "filename")
VALUES ({author_id}, '{filename}')
"""

print(sql)


INSERT INTO myboard_image
("author_id", "filename")
VALUES (1, 'face2.png')



In [96]:
v = cursor.execute(sql)

In [99]:
sql = f"""
    select filename  
    from myboard_image
    where author_id  = ( select id  from auth_user where username='{username}') 
"""
print(sql)


    select filename  
    from myboard_image
    where author_id  = ( select id  from auth_user where username='home') 



In [101]:
cursor = connection.cursor()
cursor.execute(sql)    
data = dictfetchall(cursor)    
context = {"data":data, "username":username}    
print(context)

{'data': [{'filename': 'f1.jpg'}, {'filename': 'f2.jpg'}, {'filename': 'f3.jpg'}, {'filename': 'f4.jpg'}, {'filename': 'f5.jpg'}], 'username': 'home'}


In [103]:
response = render(request,  "myboard/photolist.html", context)

In [104]:
print(response.content.decode())


home   <br>


<form action="upload" enctype="multipart/form-data" method="POST">
    <input type='hidden' name='csrfmiddlewaretoken' value='aUvBFM9bL6HLr9D7yBFMqNlWIY50DrOBVPIG7ACDXKPOyZfMV6t2IG4Tq8FMUlfb' />
      <input type="file" name="filename"/>
      <input type="submit" value="사진정송">
      <input type="hidden" name="username" value="home" >
</form>


<img src="/static/faces/home/f1.jpg" width="100" >


<img src="/static/faces/home/f2.jpg" width="100" >


<img src="/static/faces/home/f3.jpg" width="100" >


<img src="/static/faces/home/f4.jpg" width="100" >


<img src="/static/faces/home/f5.jpg" width="100" >




In [ ]:
sql = f"""
    select filename  
    from myboard_image
    where author_id  = ( select id  from auth_user where username='{username}') 
"""
cursor = connection.cursor()
cursor.execute(sql)    
data = dictfetchall(cursor)    
context = {"data":data, "username":username}    

render(request,  "myboard/photolist.html", context)

In [ ]:
file = request.FILES['file1']
filename = file._name
fp = open(settings.BASE_DIR + "/static/" + filename, "wb")
for chunk in file.chunks() :
        fp.write(chunk)
fp.close()

In [129]:
from django.core.files.uploadedfile import SimpleUploadedFile
f = open("static/people.jpg", "rb")

file_upload = SimpleUploadedFile("dog2.jpg", f.read(), 
                                 content_type="image/jpeg")
data = {
        "filename" : file_upload, "username" : "home"
    }
request = rf.post("/myboard/upload", data=data, format='multipart')

In [133]:
username = request.POST["username"]

In [131]:
file = request.FILES['filename']
filename = file._name
print(filename)
fp = open(settings.BASE_DIR + "/static/faces/" + username + "/"+ filename, "wb")
for chunk in file.chunks() :
    fp.write(chunk)
fp.close()

dog2.jpg
